In [3]:
import os
import sys

# 获取当前notebook的绝对路径
notebook_path = os.path.abspath("")

# 获取上一层目录
parent_dir = os.path.dirname(notebook_path)

# 添加路径
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [14]:
from agents import Agent, Runner
from model_provider.model_provider import MODEL_PROVIDER
from dataclasses import dataclass
from enum import Enum

class ReadMethod(Enum):
    SQL = "SQL"
    PANDAS = "PANDAS"

@dataclass
class DataReadParams:
    method: ReadMethod  # 只能是 ReadMethod.SQL 或 ReadMethod.PANDAS
    "读取方式：`SQL`或`PANDAS`"
    params: str
    "读取参数：SQL查询或文件路径"

loader_agent = Agent(
    name="loader_agent",
    instructions="""
    针对用户输入，提取数据读取参数。
    """,
    model=MODEL_PROVIDER.get_model(None),
    output_type=DataReadParams
)

In [18]:
res = await Runner.run(
    starting_agent=loader_agent,
    input='分析当月收入'
)

In [20]:
res.final_output

DataReadParams(method=<ReadMethod.SQL: 'SQL'>, params='$Month = MONTH(CURRENT_DATE())')

In [1]:
import pandas as pd

df = pd.read_csv('../test.csv')

In [10]:
from dataclasses import dataclass, field
from typing import Any, Dict

@dataclass
class Data:
    data: Any = None  # 改用 default=None
    """数据内容"""

    washed: bool = False
    """是否清洗"""

@dataclass
class AnalysisResult:
    id: str = ""
    """数据分析id"""

    text: str = ""
    """数据分析结果文本"""

@dataclass
class PictureResult:
    title: str = ""
    """图片名称"""

    path: str = ""
    """图片路径"""

    description: str = ""
    """图片描述"""

@dataclass
class AnalysisContext:
    data: Data = field(default_factory=Data)
    result: Dict[str, AnalysisResult] = field(default_factory=dict)
    pics: Dict[str, PictureResult] = field(default_factory=dict)

In [2]:
if df is not None:
    print(df)

   col1  col2
0     1     4
1     2     5
2     3     6


In [11]:
context = AnalysisContext()
context

AnalysisContext(data=Data(data=None, washed=False), result={}, pics={})

In [14]:
context.data.data is None

True

In [3]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, date
from uuid import uuid4

def generate_mixed_dataframe(num_rows=100):
    """生成包含所有主要数据类型的DataFrame"""
    
    # 基本数据类型
    data = {
        # 数值类型
        'integer': np.random.randint(-100, 100, size=num_rows),
        'float': np.random.uniform(-10, 10, size=num_rows),
        'boolean': np.random.choice([True, False], size=num_rows),
        
        # 文本类型
        'string': [f"text_{random.randint(1, 100)}" for _ in range(num_rows)],
        'category': pd.Series(np.random.choice(['A', 'B', 'C', 'D'], size=num_rows), dtype="category"),
        
        # 时间类型
        'datetime': pd.date_range(start='2020-01-01', periods=num_rows, freq='H'),
        'date': [date(2020, 1, 1) + pd.Timedelta(days=x) for x in range(num_rows)],
        'timedelta': [pd.Timedelta(seconds=x*3600) for x in range(num_rows)],
        
        # 特殊类型
        'uuid': [str(uuid4()) for _ in range(num_rows)],
        'complex': [complex(random.random(), random.random()) for _ in range(num_rows)],
        
        # 缺失值
        'null_integer': [x if x%5 != 0 else None for x in np.random.randint(0, 50, size=num_rows)],
        'null_float': [x if x%7 != 0 else None for x in np.random.uniform(0, 1, size=num_rows)],
        'null_string': [f"text_{x}" if x%3 != 0 else None for x in range(num_rows)],
        
        # 二进制数据
        'bytes': [f"sample_{x}".encode('utf-8') for x in range(num_rows)],
    }
    
    # 创建DataFrame
    df = pd.DataFrame(data)
    
    # 添加一些更复杂的数据类型
    df['mixed'] = df.apply(lambda row: 
        random.choice([row['integer'], row['float'], row['string'], row['datetime'], row['boolean']]), 
        axis=1)
    
    df['nested_dict'] = [{'a': random.randint(1, 10), 'b': random.random()} for _ in range(num_rows)]
    df['nested_list'] = [list(range(random.randint(1, 5))) for _ in range(num_rows)]
    
    return df

# 生成DataFrame
df = generate_mixed_dataframe(50)

# 保存到CSV文件
csv_file = 'test.csv'
df.to_csv(csv_file, index=False)

print(f"包含所有数据类型的DataFrame已保存到 {csv_file}")
print("\nDataFrame示例:")
print(df.head(3))

包含所有数据类型的DataFrame已保存到 test.csv

DataFrame示例:
   integer     float  boolean   string category            datetime  \
0       -7  0.299045     True  text_75        A 2020-01-01 00:00:00   
1      -47  3.703936    False  text_79        D 2020-01-01 01:00:00   
2       44 -8.244897    False  text_28        C 2020-01-01 02:00:00   

         date       timedelta                                  uuid  \
0  2020-01-01 0 days 00:00:00  da4e070f-d648-4dba-a9f8-eb1fd9699854   
1  2020-01-02 0 days 01:00:00  cb4913c9-201b-4189-8678-7b5f00f0878f   
2  2020-01-03 0 days 02:00:00  65fe915e-ca8a-41e8-99c3-116007ac2e45   

              complex  null_integer  null_float null_string        bytes  \
0  0.068539+0.823197j          24.0    0.159909        None  b'sample_0'   
1  0.568481+0.659278j          42.0    0.130279      text_1  b'sample_1'   
2  0.303114+0.097466j          36.0    0.234771      text_2  b'sample_2'   

                 mixed                        nested_dict      nested_list  
0 

/tmp/ipykernel_1465470/1170171295.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'datetime': pd.date_range(start='2020-01-01', periods=num_rows, freq='H'),


In [ ]:
df.describe()

In [1]:
text="""<think>\n好的，我现在需要分析用户提供的关于布尔类型数据的统计结果。用户给出的数据是True有27个，False有23个，名字是count，数据类型是int64。首先，我要理解这些数据代表什么。布尔
型通常是指true或false的值，所以这可能是一个分类问题，比如某个条件是否满足。接下来，我需要计算一些描述性统计指标，比如频率、百分比、均值等，来概括这些数据的特征。\n\n首先，总样本数
是True和False的总和，也就是27+23=50。这可以确认总数是否正确。然后，计算各部分的百分比，True的比例是27/50=0.54，也就是54%，False是23/50=46%。这有助于了解数据的分布是否平衡，是否有可
能存在类别不平衡的问题，这在机器学习中可能是一个重要的考虑因素。\n\n接下来，均值。由于布尔型数据通常用1代表True，0代表False，所以均值就是True的比例，也就是54%。中位数可能需要看看数
据排序后的中间值，但布尔型数据只有两个值，所以中位数可能也是1，因为当True多于False时，中间值会是1？不过实际计算中，50个数据，中间两个数是第25和26个，如果True有27个，那么前27个都是T
rue，所以中位数是True对应的1。或者需要具体看排序情况，但这里可能直接说中位数是1。\n\n然后众数就是出现次数最多的类别，True出现27次，所以众数是True。标准差的话，布尔数据的方差是p*(1-
p)，这里p=0.54，方差是0.54*0.46≈0.2484，标准差就是sqrt(0.2484)≈0.498。这说明数据在均值附近的离散程度，标准差接近0.5，说明数据离散程度较高，因为布尔数据的最大标准差是0.5，当p=0.5时
最大，所以这里接近0.5，说明分布比较分散，但稍微偏向True。\n\n接下来是否有缺失值？用户给出的数据是count列，也就是True和False的数量，但可能需要确认总样本数是否确实是50，比如可能有缺
失值的话总样本数可能更高。但用户没有提到缺失值，所以可能数据中没有缺失值。不过需要指出这一点，可能在分析中需要考虑是否存在缺失值的可能性。\n\n然后，是否有异常值？布尔数据只有两个可
能的值，所以不太会有传统的数值型异常值，但可以检查是否有极端的类别不平衡。这里True比False多14个百分点，属于中等不平衡，但不算极端，比如如果有一个类别超过90%可能更严重不平衡，但这里
54%和46%可能在多数情况下是可以接受的，不过具体还是要看应用场景。\n\n用户的需求可能是在进行数据分析或建模，需要了解数据的分布情况。例如，在机器学习中，类别平衡会影响模型表现，尤其是
在分类任务中。因此，是否需要采取上采样、下采样，或者调整类别权重，可能需要根据这个比例来决定。此外，在描述性统计中，这些指标可以帮助理解数据的基本情况，为后续分析提供基础。\n\n另外
，可能需要可视化，比如柱状图显示True和False的分布，或者饼图，但用户目前只需要文本分析。需要将结果以清晰的方式呈现，比如百分比、均值、标准差等，并解释每个指标的意义，以及可能的应用
影响。同时，提醒用户是否需要更深入的分析，比如与其他变量的交叉分析，或者在不同数据集中的分布比较等。\n\n总结来说，用户提供的数据是布尔类型，True和False的数量分别为27和23。需要计算
频率、百分比、均值、中位数、众数、标准差，评估平衡性，是否存在缺失值，以及分布的其他特征。这些信息有助于用户全面了解数据的基本特征，并为进一步的分析或建模决策提供依据。\n</think>\n
\n### **描述性分析结果：**\n\n1. **总体样本数**  \n   总共有 **50** 个布尔型数据点（True + False）。\n\n2. **频率分布**  \n   - **True**: 27 次（占比 **54%**）  \n   - **False**: 
23 次（占比 **46%**）\n\n3. **关键统计指标**  \n   - **均值（Mean）**: 0.54  \n     （布尔值以 1 表示 True，0 表示 False，均值等价于 True 的比例）  \n   - **标准差（Std）**: ≈ 
0.498  \n     （公式为 √ = √[0.54×0.46] ≈ 0.498，最大标准差为 0.5）  \n   - **中位数（Median）**: 1 （对应 True）  \n     （样本数为偶数，第 25 和 26 个值为 True）  \n   - 
**众数（Mode）**: True（出现频次最多）  \n\n4. **分布特征**  \n   - **类别平衡性**:  \n     数据接近均衡（True: 54% vs False: 46%），属于**轻度不平衡**。  \n   - **离散性**:  \n   
标准差接近最大值（0.5），说明 True 和 False 的分布较为分散，但略微倾向 True。  \n\n5. **其他观察**  \n   - **缺失值**: 未提及缺失值，总样本数为 50，假设无缺失。  \n   - 
**潜在应用场景**:  \n     - 分类任务需关注类别平衡，可能需要调整采样或损失权重。  \n     - 若布尔值代表某项属性（如用户是否购买），True 略多，但差异不算显著。\n\n---\n\n### 
**总结**  \n该布尔型数据总体分布接近 1:1，True 略占优势。均值为 
0.54，标准差接近最大值，说明数据较为分散。在需要模型训练或进一步分析时，需注意其轻度不平衡可能带来的影响。建议结合其他变量进行交叉分析（如与其他特征的相关性）以获取更深入的洞察。"""

In [3]:
import re

def remove_think_tags(text):
    """
    删除文本中<think>和</think>之间的内容
    
    参数:
        text (str): 包含<think>和</think>标签的原始文本
        
    返回:
        str: 移除了<think>标签及其内容的文本
    """
    pattern = r'<think>.*?</think>'
    return re.sub(pattern, '', text, flags=re.DOTALL).strip()

# 使用示例
cleaned_text = remove_think_tags(text)
print(cleaned_text.strip())

### **描述性分析结果：**

1. **总体样本数**  
   总共有 **50** 个布尔型数据点（True + False）。

2. **频率分布**  
   - **True**: 27 次（占比 **54%**）  
   - **False**: 
23 次（占比 **46%**）

3. **关键统计指标**  
   - **均值（Mean）**: 0.54  
     （布尔值以 1 表示 True，0 表示 False，均值等价于 True 的比例）  
   - **标准差（Std）**: ≈ 
0.498  
     （公式为 √ = √[0.54×0.46] ≈ 0.498，最大标准差为 0.5）  
   - **中位数（Median）**: 1 （对应 True）  
     （样本数为偶数，第 25 和 26 个值为 True）  
   - 
**众数（Mode）**: True（出现频次最多）  

4. **分布特征**  
   - **类别平衡性**:  
     数据接近均衡（True: 54% vs False: 46%），属于**轻度不平衡**。  
   - **离散性**:  
   
标准差接近最大值（0.5），说明 True 和 False 的分布较为分散，但略微倾向 True。  

5. **其他观察**  
   - **缺失值**: 未提及缺失值，总样本数为 50，假设无缺失。  
   - 
**潜在应用场景**:  
     - 分类任务需关注类别平衡，可能需要调整采样或损失权重。  
     - 若布尔值代表某项属性（如用户是否购买），True 略多，但差异不算显著。

---

### 
**总结**  
该布尔型数据总体分布接近 1:1，True 略占优势。均值为 
0.54，标准差接近最大值，说明数据较为分散。在需要模型训练或进一步分析时，需注意其轻度不平衡可能带来的影响。建议结合其他变量进行交叉分析（如与其他特征的相关性）以获取更深入的洞察。
